In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import all necesary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
%matplotlib inline
import seaborn as sns
import calendar
import plotly as pt
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from pylab import *
import matplotlib.patheffects as PathEffects
import gc
import descartes
import geopandas as gpd
from Levenshtein import distance
from itertools import product
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from scipy.spatial.distance import pdist, squareform
from shapely.geometry import Point, Polygon

import geoplot
from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings('ignore')


plt.rcParams['font.family'] = "Microsoft JhengHei UI Light"
plt.rcParams['font.serif'] = ["Microsoft JhengHei UI Light"]
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [27]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_csv('US.csv')
df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,Right lane blocked due to accident on I-70 Eas...,I-70 E,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,Brice Rd,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,Accident on I-75 Southbound at Exits 52 52B US...,I-75 S,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Miamisburg Centerville Rd,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   Severity               int64  
 3   Start_Time             object 
 4   End_Time               object 
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object 
 11  Street                 object 
 12  City                   object 
 13  County                 object 
 14  State                  object 
 15  Zipcode                object 
 16  Country                object 
 17  Timezone               object 
 18  Airport_Code           object 
 19  Weather_Timestamp      object 
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)       

In [29]:
df.describe()


,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,7.728394e+06,7.728394e+06,7.728394e+06,4.325632e+06,4.325632e+06,7.728394e+06,7.564541e+06,5.729375e+06,7.554250e+06,7.587715e+06,7.551296e+06,7.157161e+06,5.524808e+06
mean,2.212384e+00,3.620119e+01,-9.470255e+01,3.626183e+01,-9.572557e+01,5.618423e-01,6.166329e+01,5.825105e+01,6.483104e+01,2.953899e+01,9.090376e+00,7.685490e+00,8.407210e-03
std,4.875313e-01,5.076079e+00,1.739176e+01,5.272905e+00,1.810793e+01,1.776811e+00,1.901365e+01,2.238983e+01,2.282097e+01,1.006190e+00,2.688316e+00,5.424983e+00,1.102246e-01
min,1.000000e+00,2.455480e+01,-1.246238e+02,2.456601e+01,-1.245457e+02,0.000000e+00,-8.900000e+01,-8.900000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,3.339963e+01,-1.172194e+02,3.346207e+01,-1.177543e+02,0.000000e+00,4.900000e+01,4.300000e+01,4.800000e+01,2.937000e+01,1.000000e+01,4.600000e+00,0.000000e+00
50%,2.000000e+00,3.582397e+01,-8.776662e+01,3.618349e+01,-8.802789e+01,3.000000e-02,6.400000e+01,6.200000e+01,6.700000e+01,2.986000e+01,1.000000e+01,7.000000e+00,0.000000e+00
75%,2.000000e+00,4.008496e+01,-8.035368e+01,4.017892e+01,-8.024709e+01,4.640000e-01,7.600000e+01,7.500000e+01,8.400000e+01,3.003000e+01,1.000000e+01,1.040000e+01,0.000000e+00
max,4.000000e+00,4.900220e+01,-6.711317e+01,4.907500e+01,-6.710924e+01,4.417500e+02,2.070000e+02,2.070000e+02,1.000000e+02,5.863000e+01,1.400000e+02,1.087000e+03,3.647000e+01


In [30]:
#check to see if missing values are in same rows
df[np.logical_xor(df.End_Lat.isna(),df.End_Lng.isna()) == True]

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight


In [31]:
# Create a DataFrame with the column names and null counts
null_counts = pd.DataFrame({
    'Column': df.columns,
    'Null Count': [df[column].isnull().sum() for column in df.columns]
})

# Calculate the percentage of missing values
null_counts['Percentage Missing'] = null_counts['Null Count'] / df.shape[0] * 100
null_counts['Percentage Missing'] = null_counts['Percentage Missing'].round(2)
print(null_counts)

                   Column  Null Count  Percentage Missing
0                      ID           0                0.00
1                  Source           0                0.00
2                Severity           0                0.00
3              Start_Time           0                0.00
4                End_Time           0                0.00
5               Start_Lat           0                0.00
6               Start_Lng           0                0.00
7                 End_Lat     3402762               44.03
8                 End_Lng     3402762               44.03
9            Distance(mi)           0                0.00
10            Description           5                0.00
11                 Street       10869                0.14
12                   City         253                0.00
13                 County           0                0.00
14                  State           0                0.00
15                Zipcode        1915                0.02
16            

In [32]:
#create dataframe of variables collected at airport for airport_code with null values (i.e., no airport information)
weather_info = df[df.Airport_Code.isna()][['Airport_Code','Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)','Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition']]

In [33]:
#all records that are missing an airport_code are also missing weather information
print('Percentage null when Airport_Code is null:')
for weather in weather_info.columns:
    print('{}: {}%'.format(weather, (weather_info[weather].isna().count()/weather_info.Airport_Code.isna().count())*100))

Percentage null when Airport_Code is null:
Airport_Code: 100.0%
Weather_Timestamp: 100.0%
Temperature(F): 100.0%
Wind_Chill(F): 100.0%
Humidity(%): 100.0%
Pressure(in): 100.0%
Visibility(mi): 100.0%
Wind_Direction: 100.0%
Wind_Speed(mph): 100.0%
Precipitation(in): 100.0%
Weather_Condition: 100.0%


In [34]:
del weather_info
gc.collect()

539

In [ ]:
# Create a DataFrame with the column names and null counts
null_counts = pd.DataFrame({
    'Column': df.columns,
    'Null Count': [df[column].isnull().sum() for column in df.columns]
})

# Calculate the percentage of missing values
null_counts['Percentage Missing'] = null_counts['Null Count'] / df.shape[0] * 100
null_counts['Percentage Missing'] = null_counts['Percentage Missing'].round(2)
print(null_counts)

In [36]:
df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,Right lane blocked due to accident on I-70 Eas...,I-70 E,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,Brice Rd,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,Accident on I-75 Southbound at Exits 52 52B US...,I-75 S,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Miamisburg Centerville Rd,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day


In [37]:
df['Severity'].value_counts()

Severity
2    6156981
3    1299337
4     204710
1      67366
Name: count, dtype: int64

In [38]:
df.dropna(subset=['Street', 'City', 'Zipcode'], inplace=True)

In [39]:
tznullindex = np.array(df[df.Timezone.isna()].index)
tznulllatlng = df.loc[tznullindex, ['Start_Lat','Start_Lng']]

In [40]:
#fill in missing time zones
from timezonefinder import TimezoneFinder
obj = TimezoneFinder()
for i in tznullindex:
    timezone = obj.timezone_at(lng=tznulllatlng.loc[i,'Start_Lng'], lat=tznulllatlng.loc[i,'Start_Lat'])
    df.loc[i,'Timezone'] = timezone

In [60]:
df=df.convert_dtypes()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7715358 entries, 0 to 7728393
Data columns (total 49 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   ID                           string        
 1   Source                       string        
 2   Severity                     Int64         
 3   Start_Time                   string        
 4   End_Time                     string        
 5   Start_Lat                    Float64       
 6   Start_Lng                    Float64       
 7   End_Lat                      Float64       
 8   End_Lng                      Float64       
 9   Distance(mi)                 Float64       
 10  Description                  string        
 11  Street                       string        
 12  City                         string        
 13  County                       string        
 14  State                        string        
 15  Zipcode                      string        
 16  Count

In [52]:
# Remove fractional seconds from 'Start_Time' column
df['Start_Time'] = df['Start_Time'].str.replace(r'\.\d+', '', regex=True)

# Now, try parsing the datetime again
df['datetime_start_parsed'] = pd.to_datetime(df['Start_Time'], format="%Y-%m-%d %H:%M:%S")


In [58]:
df['End_Time'] = df['End_Time'].str.replace(r'\.\d+', '', regex=True)

df['datetime_end_parsed'] = pd.to_datetime(df['End_Time'], format="%Y-%m-%d %H:%M:%S")


In [59]:
df['Weather_Timestamp'] = df['Weather_Timestamp'].str.replace(r'\.\d+', '', regex=True)

df['datetime_weathertime_parsed'] = pd.to_datetime(df['Weather_Timestamp'], format="%Y-%m-%d %H:%M:%S")

In [61]:
#remove start/end/weather time (already parsed)
df = df.drop(['ID','Start_Time','End_Time', 'Weather_Timestamp'], axis=1)

In [62]:
#copy values from start lat/lng to end lat/lng for null values
latnull = np.array(df[np.isnan(df.End_Lat)].index)
df.loc[latnull,'End_Lat'] = df.loc[latnull,'Start_Lat']
lngnull = np.array(df[np.isnan(df.End_Lng)].index)
df.loc[lngnull,'End_Lng'] = df.loc[lngnull,'Start_Lng']

#replace missing wind chill data with temperature
wcnull = np.array(df[np.isnan(df['Wind_Chill(F)'])].index)
df.loc[wcnull,'Wind_Chill(F)'] = df.loc[i,'Temperature(F)']

#replace missing temperature data with wind chill
tempnull = np.array(df[np.isnan(df['Temperature(F)'])].index)
df.loc[wcnull,'Temperature(F)'] = df.loc[i,'Wind_Chill(F)']

#replace missing data for when the weather was collected based on end time of the accident
weathertimenull = np.array(df[np.isnan(df.datetime_weathertime_parsed)].index)
df.loc[weathertimenull,'datetime_weathertime_parsed'] = df.loc[weathertimenull,'datetime_end_parsed']

In [63]:
#N/A Precipitation must mean virtually no rain
df[df.Weather_Condition == 'N/A Precipitation']['Precipitation(in)'].mean()

0.008632824665213331

In [64]:
#replace N/A Precipitation with NaN to be imputed with most frequent
df.Weather_Condition = df.Weather_Condition.replace('N/A Precipitation', np.NaN)

In [69]:
#variables to impute with median or mode strategy
median_impute = ['Temperature(F)','Wind_Chill(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Speed(mph)','Precipitation(in)']
mode_impute = ['Wind_Direction','Weather_Condition','Sunrise_Sunset','Civil_Twilight', 'Nautical_Twilight','Astronomical_Twilight']

In [70]:
#mode impute (SimpleImputer for loop is too slow)
wdnull = np.array(df[df.Wind_Direction.isna()].index)
df.loc[wdnull,'Wind_Direction'] = 'CALM'

wcnull = np.array(df[df.Weather_Condition.isna()].index)
df.loc[wcnull,'Weather_Condition'] = 'Fair'

ssnull = np.array(df[df.Sunrise_Sunset.isna()].index)
df.loc[ssnull,'Sunrise_Sunset'] = 'Day'

ctnull = np.array(df[df.Civil_Twilight.isna()].index)
df.loc[ssnull,'Civil_Twilight'] = 'Day'

ntnull = np.array(df[df.Nautical_Twilight.isna()].index)
df.loc[ssnull,'Nautical_Twilight'] = 'Day'

atnull = np.array(df[df.Astronomical_Twilight.isna()].index)
df.loc[atnull,'Astronomical_Twilight'] = 'Day'

In [71]:
#impute variables with median values
import numpy as np
from sklearn.impute import SimpleImputer

imputer1 = SimpleImputer(missing_values=np.nan, strategy='median')
df_median_fit = imputer1.fit_transform(df[median_impute])
df_median = imputer1.transform(df_median_fit)

df[median_impute] = pd.DataFrame(df_median)

In [73]:
# Specify columns to check for null values
columns_to_check = ['Description', 'Airport_Code', 'Precipitation(in)','Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)']

# Drop rows with null values in specified columns
df.dropna(subset=columns_to_check, inplace=True)

In [76]:
df.to_csv('cleaned_USA.csv', index=False)


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7700098 entries, 0 to 7715357
Data columns (total 45 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   Source                       string        
 1   Severity                     Int64         
 2   Start_Lat                    Float64       
 3   Start_Lng                    Float64       
 4   End_Lat                      Float64       
 5   End_Lng                      Float64       
 6   Distance(mi)                 Float64       
 7   Description                  string        
 8   Street                       string        
 9   City                         string        
 10  County                       string        
 11  State                        string        
 12  Zipcode                      string        
 13  Country                      string        
 14  Timezone                     string        
 15  Airport_Code                 string        
 16  Tempe